Game Sales data preparation

In [ ]:
#Importing Required Libraries
import time
import requests
import json
import pymongo
from bs4 import BeautifulSoup
import urllib.request
import re

In [ ]:
#Passing the header value
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

#An open session carries the cookies and allows you to make post requests
session_requests = requests.session()

#defining the url - All features have been selected, no filters applied
base_url = "https://www.vgchartz.com/games/games.php?page="
url_specifics = "&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1"

#Creating a mongodb connection
client = pymongo.MongoClient("mongodb://localhost:27017/")

#Selecting the database
db = client["Game_Sales"]

#Dropping the collection if it already exists
#if "Overview" in db.list_collection_names():
#    db.drop_collection("Overview")

#Selecting the collection
overview = db["Overview"]

#defining the url - All features have been selected, no filters applied
base_url = "https://www.vgchartz.com/games/games.php?page="
url_specifics = "&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1"

#Defining a variable for the page number
i = 1

#Observing the page source for page 1 to identify the number of pages to be traversed
url = base_url + str(i) + url_specifics
web_page = session_requests.get(url, headers = headers)
soup = BeautifulSoup(web_page.content, "html.parser")
game_elements = soup.find('div',{'id':'generalBody'})
pages = game_elements.find('th',{'style':'text-align:right;'})
pages = pages.find_all('span')[6]
pages = pages.find('a')['href']
num_pages = int(re.search(r'page=(\d+)', pages).group(1), base = 16)

#Downloading the pages for further processing
for i in range(num_pages):
    page_number = i + 1
    
    #Concatenating the URL to be loaded
    url = base_url + str(page_number) + url_specifics
    
    #Passing the URL for sending the GET request
    web_page = session_requests.get(url, headers = headers)
    
    #Parse the response
    soup = BeautifulSoup(web_page.content, "html.parser")    
    
    with open(f"game_sales_overview_{i + 1}.html", "w", encoding = "utf-8") as f:
        f.write(str(soup))

#Creating a loop to traverse through the downloaded pages and extract the required content
for i in range(i, num_pages):
    page_number = i
    
    with open(f"game_sales_overview_{i}.html", "r", encoding = "utf-8") as f:
        content = f.read()
        
    soup = BeautifulSoup(content, "html.parser")    
    
    #Finding the body element containing all the game details
    game_elements = soup.find('div',{'id':'generalBody'})
    
    #Finding the list of games on the page based on the style
    games_list = game_elements.find_all('tr',{'style':'background-image:url(../imgs/chartBar_large.gif); height:70px'})
    games_list1 = game_elements.find_all('tr',{'style':'background-image:url(../imgs/chartBar_alt_large.gif); height:70px'})
    
    #Appending all game entries to an element
    for k in range(len(games_list1)):
        games_list.append(games_list1[k])
    
    #Traversing for each game present in the list
    for k in range(len(games_list)):
        #Storing the element to be inspected
        game = games_list[k]
        
        #Storing the Game Rank
        game_rank = game.find('td').text.strip()
        
        #Finding the element containing the Game Name and URL
        game_name_url = game.find('td',{'style':'font-size:12pt;'})
        
        #Finding the Game Name
        game_name = game_name_url.text.strip()
        
        #Storing the link of the game for further data extraction
        game_url = game_name_url.find('a', href = True)['href']
        
        #Finding all the elements with id 'td'
        game_details = game.find_all('td')
        
        #Storing the console
        game_console = game_details[3].find('img')['alt']
        
        #Storing the publisher
        game_publisher = game_details[4].text.strip()
        
        #Storing the developer
        game_developer = game_details[5].text.strip()
        
        #Storing the website game score
        game_vg_score = game_details[6].text.strip()
        
        #Storing the critic score
        game_critic_score = game_details[7].text.strip()
        
        #Storing the user score
        game_user_score = game_details[8].text.strip()
        
        #Storing the total shipped
        game_total_shipped = game_details[9].text.strip()
        
        #Storing the total sales
        game_total_sales = game_details[10].text.strip()
        
        #Storing the NA sales
        game_NA_sales = game_details[11].text.strip()
        
        #Storing the PAL sales
        game_PAL_sales = game_details[12].text.strip()
        
        #Storing the Japan sales
        game_JP_sales = game_details[13].text.strip()
        
        #Storing the other sales
        game_other_sales = game_details[14].text.strip()
        
        #Storing the Release Date
        game_release_date = game_details[15].text.strip()
        
        #Storing the Last Update Date
        game_last_update = game_details[16].text.strip()
        
        #Opening each game's web page to download further details
        #Passing the URL for sending the GET request
        game_page = session_requests.get(game_url, headers = headers)
        
        #Parse the response
        soup = BeautifulSoup(game_page.content, "html.parser")
        
        #Defining a list to store the game releases
        game_releases = []
        
        #Storing the different releases of the game
        game_body = soup.find('div',{'id' : 'gameBody'})
        game_body = game_body.find_all('li')
        if game_body:
            for release in game_body:
                game_releases.append(release.text.strip())
        else:
            game_releases.append("NA")
        
        #Finding the element containing Genre
        game_details = soup.find('div',{'id' : 'gameGenInfoBox'})
        
        #Storing the genre of the game
        game_genre = game_details.find('h2', text = 'Genre')
        game_genre = game_genre.find_next_sibling('p').text.strip()
        
        #Creating a dictionary containing the data
        game_overview_data = {
            "Rank" : game_rank,
            "Name" : game_name,
            "URL" : game_url,
            "Console" : game_console,
            "Genre" : game_genre,
            "Publisher" : game_publisher,
            "Developer" : game_developer,
            "Releases" : game_releases,
            "VG_score" : game_vg_score,
            "Critic_score" : game_critic_score,
            "User_score" : game_user_score,
            "Total_shipped" : game_total_shipped,
            "Total_sales" : game_total_sales,
            "NA_sales" : game_NA_sales,
            "PAL_sales" : game_PAL_sales,
            "JP_sales" : game_JP_sales,
            "Other_sales" : game_other_sales,
            "Release_date" : game_release_date,
            "Last_update": game_last_update
        }
        
        #Inserting the document into the collection
        overview.insert_one(game_overview_data)
    
    #Printing the status
    print(f"Data inserted from page number {page_number} for {len(games_list)} games")

#Creating indexes on selected features
overview.create_index("Rank", unique = True)
overview.create_index("Name", unique = False)
overview.create_index("Console", unique = False)
overview.create_index("Genre", unique = False)
overview.create_index("Developer", unique = False)
overview.create_index("Publisher", unique = False)

#Closing the connection
client.close()
